In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [ ]:
# Set the project name (this is the folder name where the output files will be stored)
custom_data_folder = 'amsterdam_custom_samples'

In [ ]:
## Number of TFRecords in a single file
batch_size = 100

In [ ]:
serval_data_folder = "../data"

In [ ]:
input_selected_classes_filepath = serval_data_folder + '/' + custom_data_folder + '/csv_files/input_selected_classes.csv'
output_class_mapping_filepath   = serval_data_folder + '/' + custom_data_folder + '/csv_files/output_class_mapping.csv' 
output_tfrecords_train_path     = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/train'
output_tfrecords_eval_path      = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/eval'

tfrecords_train_search_string   = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/train_*.tfrecord'
tfrecords_eval_search_string    = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/eval_*.tfrecord'

In [ ]:
df_input_selected_classes = pd.read_csv(input_selected_classes_filepath, sep=";")
print('[INFO]: df_input_selected_classes')
print(df_input_selected_classes)

In [ ]:
## Create class mapping to 1:20

## First validate that there are no duplicates in the input file
assert(not any(df_input_selected_classes.label.duplicated())) ##, "df_input_selected_classes.label is expected to have unique labels!")

## Create class mapping
df_output_class_mapping = df_input_selected_classes.loc[df_input_selected_classes.enable == 1].copy(deep=True)

del df_output_class_mapping['enable']
df_output_class_mapping['old_label'] = df_output_class_mapping['label']
df_output_class_mapping['label'] = list(range(0, len(df_output_class_mapping)))
df_output_class_mapping['count_train'] = 0
df_output_class_mapping['count_eval'] = 0

print('[INFO]: df_output_class_mapping')
print(df_output_class_mapping)

In [ ]:
def createTFRecord(example_old, new_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [ ]:
def getNewLabels(df_label_mapping, old_labels):
    return(df_label_mapping.loc[df_label_mapping.old_label.isin(old_labels)].label.values)

In [ ]:
def getAllTFRecords(tfrecords_search_string, df_class_mapping, count_type):
    tfrecords = {}
    
    ## Get files
    files = gfile.Glob(tfrecords_search_string)
    print('[INFO]: Processing ', len(list(files)), ' files from directory: ', tfrecords_search_string, sep='')
    
    for file in files:
        records = list(tf.python_io.tf_record_iterator(path = file))
        print('[INFO]: Processing: ', len(records), ' tfrecords from file: ', file, sep='')

        for string_record in records:
            old_tfrecord = tf.train.SequenceExample()
            old_tfrecord.ParseFromString(string_record)

            ## Get labels and find for each label the m-code in class_label_indices_old
            old_labels = old_tfrecord.context.feature['labels'].int64_list.value
            new_labels = getNewLabels(df_class_mapping, old_labels)
            if len(new_labels) == 0:
                ## Skip files without valid labels
                continue
                
            new_tfrecord = createTFRecord(old_tfrecord, new_labels)

            ## Update count of found labels
            df_class_mapping[count_type] = df_class_mapping[count_type] + np.where(df_class_mapping.old_label.isin(old_labels), 1, 0)

            ## Store new record in dict
            tfrecords[len(tfrecords)] = new_tfrecord
            
    return({'tfrecords':tfrecords, 'df_class_mapping':df_class_mapping})

In [ ]:
def writeTFRecordsInBatches(tfrecords, batch_size, output_directory):
    ## Generate random permutation without replace
    permutation = random.sample(range(0, len(tfrecords)), len(tfrecords))
    
    batch_index = 0
    tfrecords_remain_in_batch = -1
    for index in permutation:
        if tfrecords_remain_in_batch == 0:
            batch_writer.close()

        if tfrecords_remain_in_batch <= 0:
            ## Create new batch file
            tfrecords_remain_in_batch = batch_size
            batch_index = batch_index + 1
            batch_file_name = str(output_directory + '/sample_' + str(batch_index) + '.tfrecord')
            batch_writer = tf.python_io.TFRecordWriter(batch_file_name)

        ## Write to file
        batch_writer.write(tfrecords[index].SerializeToString())
        tfrecords_remain_in_batch = tfrecords_remain_in_batch - 1
        
    batch_writer.close()
    return(True)

In [ ]:
## Read all train records
tfrecords_train = getAllTFRecords(tfrecords_train_search_string, df_output_class_mapping.copy(deep=True), 'count_train')

## Write batches
writeTFRecordsInBatches(tfrecords_train['tfrecords'], batch_size, output_tfrecords_train_path)

## Create ouput
df_output_class_mapping = tfrecords_train['df_class_mapping']
print(df_output_class_mapping)

In [ ]:
## Read all eval records
tfrecords_eval = getAllTFRecords(tfrecords_eval_search_string, df_output_class_mapping.copy(deep=True), 'count_eval')

## Write batches
writeTFRecordsInBatches(tfrecords_eval['tfrecords'], batch_size, output_tfrecords_eval_path)

## Create ouput
df_output_class_mapping = tfrecords_eval['df_class_mapping']
print(df_output_class_mapping)

In [ ]:
## Write new labels and descriptions back to csv_file
df_output_class_mapping['display_name'] = df_output_class_mapping.description
df_output_class_mapping['index'] = df_output_class_mapping.label
df_output_class_mapping.to_csv(output_class_mapping_filepath, sep = ";")